# Dependencies

In [5]:
%load_ext autoreload
%autoreload 2
import nltk
import ipywidgets
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
tqdm.pandas()
# from pandarallel import pandarallel
# pandarallel.initialize(progress_bar=True)
import torch
import datetime
import os
import datasets_handler
import evaluation_metrics
import zeroberto
from zeroberto import ZeroBERTo
# import datasets
# from datasets import Dataset
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer
import gc

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/alealcoforado/Documents/Projetos/ZeroBERTo/zeroberto corrigido/zeroberto/datasets_handler.py:159: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


# Dataset

In [10]:
# which_dataset = 'folhauol' 
# which_dataset = 'bbc-news'
which_dataset = 'ag_news'
which_dataset = 'imdb'


hyp_template = "{}"
# hyp_template = "O tema principal deste texto é {}."
# hyp_template = "this text is about {}."
# hyp_template = "this article is about {}."

raw_data, data_col, class_col = datasets_handler.getDataset(which_dataset)
classes_list = list(raw_data[class_col].unique())
print(classes_list)

Found cached dataset imdb (/Users/alealcoforado/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/3 [00:00<?, ?it/s]

['text', 'label']


# ZeroBERTo

## Parameters

In [12]:
# max_inferences = len(train) #estava em 5000
max_inferences = 5000

# zeroshot_method = "probability_threshold"
probability_goal = 0.9
top_n_goal = 8

zeroshot_method = "dotproduct"  
# zeroshot_method = "kmeans"  

zeroshot_config = {
    'dataset':which_dataset,
    'class_col':class_col,
    'data_col':data_col,
    'split':"zeroshot",
    'method':zeroshot_method,
    'prob_goal':probability_goal,
    'top_n_goal':top_n_goal,
    'max_inferences':max_inferences,
    'classes':classes_list,#list(dict_classes_folha.values())
    'template': hyp_template,
    'random_state':422,
    'trainings_done':0
}
zeroshot_config

{'dataset': 'imdb',
 'class_col': 'class',
 'data_col': 'text',
 'split': 'zeroshot',
 'method': 'dotproduct',
 'prob_goal': 0.9,
 'top_n_goal': 8,
 'max_inferences': 5000,
 'classes': ['text', 'label'],
 'template': '{}',
 'random_state': 422,
 'trainings_done': 0}

In [13]:
train_data = raw_data.sample(zeroshot_config['max_inferences'],random_state=zeroshot_config['random_state']).sort_index()
len(train_data)

5000

## Model

In [14]:
model = ZeroBERTo(classes_list=zeroshot_config['classes'],hypothesis_template=zeroshot_config['template'],
                  train_dataset=train_data,labeling_method=zeroshot_config['method'])

In [15]:
model.embeddingModel

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [16]:
# model.embeddingModel = new_embeddingModel

## Inference

In [62]:
results = zeroberto.runZeroberto(model,train_data[data_col].to_list(),zeroshot_config)  ##X## COMENTAR AQUI 

Preds: 100  - Total time: 7.37 seconds - ETA: 6.1 minutes
Preds: 200  - Total time: 14.29 seconds - ETA: 6.0 minutes
Preds: 300  - Total time: 21.44 seconds - ETA: 6.0 minutes
Preds: 400  - Total time: 29.21 seconds - ETA: 6.1 minutes
Preds: 500  - Total time: 36.97 seconds - ETA: 6.2 minutes
Preds: 600  - Total time: 44.15 seconds - ETA: 6.1 minutes
Preds: 700  - Total time: 51.02 seconds - ETA: 6.1 minutes
Preds: 800  - Total time: 58.01 seconds - ETA: 6.0 minutes
Preds: 900  - Total time: 64.96 seconds - ETA: 6.0 minutes
Preds: 1000  - Total time: 72.57 seconds - ETA: 6.0 minutes
Preds: 1100  - Total time: 79.92 seconds - ETA: 6.1 minutes
Preds: 1200  - Total time: 87.22 seconds - ETA: 6.1 minutes
Preds: 1300  - Total time: 94.92 seconds - ETA: 6.1 minutes
Preds: 1400  - Total time: 102.34 seconds - ETA: 6.1 minutes
Preds: 1500  - Total time: 109.5 seconds - ETA: 6.1 minutes
Preds: 1600  - Total time: 116.19 seconds - ETA: 6.1 minutes
Preds: 1700  - Total time: 123.33 seconds - ETA:

## Evaluation

In [63]:
df_results = model.evaluateLabeling(results)

top 1: {'accuracy': 1.0}
top 2: {'accuracy': 1.0}
top 3: {'accuracy': 1.0}
top 4: {'accuracy': 0.875}
top 5: {'accuracy': 0.8}
top 6: {'accuracy': 0.8333333333333334}
top 7: {'accuracy': 0.8571428571428571}
top 8: {'accuracy': 0.875}
top 9: {'accuracy': 0.8333333333333334}
top 10: {'accuracy': 0.8}
top 11: {'accuracy': 0.7727272727272727}
top 12: {'accuracy': 0.75}
top 13: {'accuracy': 0.7692307692307693}
top 14: {'accuracy': 0.7857142857142857}
top 15: {'accuracy': 0.8}
top 16: {'accuracy': 0.75}
top 5000: {'accuracy': 0.5662}


In [64]:
try: print(zeroshot_metrics)
except: pass
zeroshot_metrics = evaluation_metrics.get_metrics(df_results['prediction_code'].to_list(),df_results['class_code'].to_list())
print(zeroshot_metrics)

{'weighted': [{'accuracy': 0.567}, {'precision': 0.5675149000253721}, {'recall': 0.567}, {'f1': 0.5666831361790388}], 'macro': [{'accuracy': 0.567}, {'precision': 0.5674436559967043}, {'recall': 0.5672463275556059}, {'f1': 0.5667708217647136}]}
{'weighted': [{'accuracy': 0.5662}, {'precision': 0.5788214975324903}, {'recall': 0.5662}, {'f1': 0.5448947200842807}], 'macro': [{'accuracy': 0.5662}, {'precision': 0.5790837005029481}, {'recall': 0.5643024195793722}, {'f1': 0.5440095148753178}]}


## Save Results

In [65]:
save_time = evaluation_metrics.saveZeroshotResults(zeroshot_config,df_results)

predictions_and_probabilities_test_2023_03_09__23_25_37.csv
zeroshot_config_test_2023_03_09__23_25_37.csv


# ST Training

In [66]:
use_zeroshot_previous_step = True
n = 8
top_n = 8
zeroshot_config['top_n'] = top_n
zeroshot_config['n_examples'] = n
zeroshot_config['exec_time'] = save_time

zeroshot_config['st_train_epochs'] = 10
zeroshot_config['st_train_batch_size'] = 20


split = "zeroshot" if use_zeroshot_previous_step else "fewshot"


In [67]:
zeroshot_previous_data = datasets_handler.getZeroshotPreviousData(which_dataset,class_col,top_n=top_n,exec_time=zeroshot_config['exec_time'])
raw_data_final, zeroshot_config['new_class_col'] = datasets_handler.mergeLabelingToDataset(raw_data,zeroshot_previous_data,class_col)

In [68]:
df_train, df_test = datasets_handler.splitDataset(raw_data_final,zeroshot_config)
train_dataset,test_dataset = datasets_handler.buildDatasetDict(df_train,df_test)
train_documents = datasets_handler.splitDocuments(df_train[data_col])
len(train_documents)

177

## Treinamento do Corpo do Modelo -- não descomentar

In [69]:
# model.fit(train_documents,batch_size=zeroshot_config['st_train_batch_size'],epochs=zeroshot_config['st_train_epochs'])

# Fine-Tuning (SetFit)

## Model

In [70]:
zeroshot_config['setfit_model'] = 'sentence-transformers/stsb-xlm-r-multilingual'

setfit_model = SetFitModel.from_pretrained(zeroshot_config['setfit_model'],
                                    use_differentiable_head=True,
                                    head_params={"out_features":len(zeroshot_config['classes'])})

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [71]:
setfit_model.model_body = model.embeddingModel

In [72]:
batch_size = 8
num_text_pairs = 15
num_epochs = 30

zeroshot_config["batch_size"] = batch_size
zeroshot_config["num_pairs"] = num_text_pairs
zeroshot_config["num_epochs"] = num_epochs
# Create trainer
trainer = SetFitTrainer(
    model=setfit_model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    loss_class=CosineSimilarityLoss,
    batch_size=zeroshot_config["batch_size"],
    num_iterations=zeroshot_config["num_pairs"], # Number of text pairs to generate for contrastive learning
    num_epochs=zeroshot_config["num_epochs"], # Number of epochs to use for contrastive learning
    column_mapping = {data_col: "text", 'class_code': "label"} # NÃO mudar
)
zeroshot_config

{'dataset': 'imdb',
 'class_col': 'class',
 'data_col': 'text',
 'split': 'zeroshot',
 'method': 'dotproduct',
 'prob_goal': 0.9,
 'top_n_goal': 8,
 'max_inferences': 5000,
 'classes': ['text', 'label'],
 'template': '{}',
 'random_state': 422,
 'trainings_done': 1,
 'top_n': 8,
 'n_examples': 8,
 'exec_time': '2023_03_09__23_25_37',
 'st_train_epochs': 10,
 'st_train_batch_size': 20,
 'new_class_col': 'new_class',
 'setfit_model': 'sentence-transformers/stsb-xlm-r-multilingual',
 'batch_size': 8,
 'num_pairs': 15,
 'num_epochs': 30}

In [76]:
%%time
# Train and evaluate
# trainer.freeze() # Freeze the head
# trainer.train() # Train only the body

# Unfreeze the head and freeze the body -> head-only training
trainer.unfreeze(keep_body_frozen=True)

# Unfreeze the head and unfreeze the body -> end-to-end training
# trainer.unfreeze(keep_body_frozen=False)

trainer.train(
    num_epochs=zeroshot_config["num_epochs"], # The number of epochs to train the head or the whole model (body and head)
    batch_size=zeroshot_config["batch_size"],
    body_learning_rate=1e-5, # The body's learning rate
    learning_rate=1e-2, # The head's learning rate
    l2_weight=0.1, # Weight decay on **both** the body and head. If `None`, will use 0.01.
)

zeroshot_config['trainings_done'] += 1

print(zeroshot_config)

Applying column mapping to training dataset
The `max_length` is `None`. Using the maximum acceptable length according to the current model body: 128.


Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

{'dataset': 'imdb', 'class_col': 'class', 'data_col': 'text', 'split': 'zeroshot', 'method': 'dotproduct', 'prob_goal': 0.9, 'top_n_goal': 8, 'max_inferences': 5000, 'classes': ['text', 'label'], 'template': '{}', 'random_state': 422, 'trainings_done': 3, 'top_n': 8, 'n_examples': 8, 'exec_time': '2023_03_09__23_25_37', 'st_train_epochs': 10, 'st_train_batch_size': 20, 'new_class_col': 'new_class', 'setfit_model': 'sentence-transformers/stsb-xlm-r-multilingual', 'batch_size': 8, 'num_pairs': 15, 'num_epochs': 30}
CPU times: user 2min 15s, sys: 6min 12s, total: 8min 28s
Wall time: 3min 11s


In [74]:
gc.collect()

y_pred = zeroberto.getPredictions(trainer)

Running predictions on 5000 sentences.


## Evaluation

In [75]:
try: print(setfit_all_metrics)
except: pass
setfit_all_metrics = evaluation_metrics.get_metrics(y_pred ,test_dataset["class_code"])
print(setfit_all_metrics)

{'weighted': [{'accuracy': 0.6246}, {'precision': 0.6480520344678443}, {'recall': 0.6246}, {'f1': 0.6091207925039489}], 'macro': [{'accuracy': 0.6246}, {'precision': 0.6480520344678444}, {'recall': 0.6246}, {'f1': 0.6091207925039489}]}
{'weighted': [{'accuracy': 0.5736}, {'precision': 0.693523199224561}, {'recall': 0.5736}, {'f1': 0.4954318119166699}], 'macro': [{'accuracy': 0.5736}, {'precision': 0.693523199224561}, {'recall': 0.5736}, {'f1': 0.49543181191666996}]}


In [41]:
# new_embeddingModel = trainer.model.model_body

In [42]:
setfit_exec_time  = evaluation_metrics.saveResults(zeroshot_config,setfit_all_metrics)
print(zeroshot_config)

metrics_setfit_2023_03_09__14_05_21.csv
config_setfit_2023_03_09__14_05_21.csv
{'dataset': 'ag_news', 'class_col': 'class', 'data_col': 'text', 'split': 'zeroshot', 'method': 'dotproduct', 'prob_goal': 0.9, 'top_n_goal': 8, 'max_inferences': 5000, 'classes': ['business', 'science and technology', 'sports', 'world'], 'template': '{}', 'random_state': 422, 'trainings_done': 1, 'top_n': 8, 'n_examples': 8, 'exec_time': '2023_03_09__13_53_41', 'st_train_epochs': 10, 'st_train_batch_size': 20, 'new_class_col': 'new_class', 'setfit_model': 'sentence-transformers/stsb-xlm-r-multilingual', 'batch_size': 8, 'num_pairs': 15, 'num_epochs': 30}


In [43]:
model_path = "/Users/alealcoforado/Documents/Projetos/Modelos/{exec}".format(exec=setfit_exec_time)
print(model_path)
trainer.model._save_pretrained(save_directory=model_path)

/Users/alealcoforado/Documents/Projetos/Modelos/2023_03_09__14_05_21


# ZeroBERTo 2nd run

In [ ]:
model = ZeroBERTo(classes_list=zeroshot_config['classes'],hypothesis_template=zeroshot_config['template'],
                  train_dataset=train_data,labeling_method=zeroshot_config['method'],embeddingModel=new_embeddingModel)

In [ ]:
results = zeroberto.runZeroberto(model,train_data['text'],zeroshot_config)  ##X## COMENTAR AQUI 


Preds: 100  - Total time: 9.34 seconds - ETA: 3.1 minutes
Preds: 200  - Total time: 18.15 seconds - ETA: 3.0 minutes
Preds: 300  - Total time: 27.15 seconds - ETA: 3.0 minutes
Preds: 400  - Total time: 35.55 seconds - ETA: 3.0 minutes
Preds: 500  - Total time: 43.84 seconds - ETA: 2.9 minutes
Preds: 600  - Total time: 52.08 seconds - ETA: 2.9 minutes
Preds: 700  - Total time: 60.36 seconds - ETA: 2.9 minutes
Preds: 800  - Total time: 66.73 seconds - ETA: 2.8 minutes
Preds: 900  - Total time: 72.23 seconds - ETA: 2.7 minutes
Preds: 1000  - Total time: 77.5 seconds - ETA: 2.6 minutes
Preds: 1100  - Total time: 82.88 seconds - ETA: 2.5 minutes
Preds: 1200  - Total time: 94.68 seconds - ETA: 2.6 minutes
Preds: 1300  - Total time: 114.98 seconds - ETA: 2.9 minutes
Preds: 1400  - Total time: 129.69 seconds - ETA: 3.1 minutes
Preds: 1500  - Total time: 142.77 seconds - ETA: 3.2 minutes
Preds: 1600  - Total time: 150.28 seconds - ETA: 3.1 minutes
Preds: 1700  - Total time: 155.36 seconds - ETA

In [ ]:
df_results = model.evaluateLabeling(results)
all_metrics = evaluation_metrics.get_metrics(df_results['prediction_code'].to_list(),df_results['class_code'].to_list())
zeroshot_config['trainings_done'] += 1
save_time = evaluation_metrics.saveZeroshotResults(zeroshot_config,df_results)


top 1: {'accuracy': 0.5}
top 2: {'accuracy': 0.75}
top 3: {'accuracy': 0.8333333333333334}
top 4: {'accuracy': 0.75}
top 5: {'accuracy': 0.75}
top 6: {'accuracy': 0.7083333333333334}
top 7: {'accuracy': 0.75}
top 8: {'accuracy': 0.78125}
top 9: {'accuracy': 0.7777777777777778}
top 10: {'accuracy': 0.8}
top 11: {'accuracy': 0.7954545454545454}
top 12: {'accuracy': 0.8125}
top 13: {'accuracy': 0.8269230769230769}
top 14: {'accuracy': 0.8392857142857143}
top 15: {'accuracy': 0.8333333333333334}
top 16: {'accuracy': 0.828125}
top 2000: {'accuracy': 0.5915}
predictions_and_probabilities_test_2023_02_17__07_30_06.csv
zeroshot_config_test_2023_02_17__07_30_06.csv


In [ ]:
zeroshot_config['exec_time'] = save_time

zeroshot_previous_data = datasets_handler.getZeroshotPreviousData(which_dataset,class_col,top_n=top_n,exec_time=zeroshot_config['exec_time'])
raw_data_final, zeroshot_config['new_class_col'] = datasets_handler.mergeLabelingToDataset(raw_data,zeroshot_previous_data,class_col)

In [ ]:
df_train, df_test = datasets_handler.splitDataset(raw_data_final,zeroshot_config)
train_dataset,test_dataset = datasets_handler.buildDatasetDict(df_train,df_test)

In [ ]:
trainer.train_dataset = train_dataset
trainer.test_dataset = test_dataset

trainer.unfreeze(keep_body_frozen=False)

trainer.train(
    num_epochs=zeroshot_config["num_epochs"], # The number of epochs to train the head or the whole model (body and head)
    batch_size=zeroshot_config["batch_size"],
    body_learning_rate=1e-5, # The body's learning rate
    learning_rate=1e-2, # The head's learning rate
    l2_weight=0.1, # Weight decay on **both** the body and head. If `None`, will use 0.01.
)


Applying column mapping to training dataset
The `max_length` is `None`. Using the maximum acceptable length according to the current model body: 128.


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
gc.collect()

y_pred = zeroberto.getPredictions(trainer)

Running predictions on 2000 sentences.


In [ ]:
all_metrics = evaluation_metrics.get_metrics(y_pred ,test_dataset["class_code"])
print(all_metrics)

{'weighted': [{'accuracy': 0.6045}, {'precision': 0.6950742046267486}, {'recall': 0.6045}, {'f1': 0.5852444221212971}], 'macro': [{'accuracy': 0.6045}, {'precision': 0.6950742046267486}, {'recall': 0.6045}, {'f1': 0.5852444221212971}]}
